In [14]:
import numpy as np
import matplotlib.pyplot as plt

Training set

In [15]:
# T = 50
# Nt = 50
# dt = T/Nt
# x0 = 1
# y0 = 0.1
# rho_v = np.arange(0.0, 0.1, 0.02)
# alpha_v = np.arange(0.0, 0.1, 0.02)
# beta_v = np.arange(0.0, 0.1, 0.02)
# gamma_v = np.arange(0.0, 0.1, 0.02)
# delta_v = np.arange(0.0, 0.1, 0.02)
# n_tot = 4**5
# k=0
# input_list = []
# output_list =[]

# for rho in rho_v:
#     for alpha in alpha_v:
#         for beta in beta_v:
#             for gamma in gamma_v:
#                 for delta in delta_v:
#                     k += 1
#                     # print('Advancing: '+str(k/n_tot*100)+'%')
#                     x_ex = np.zeros((Nt+1,2))
#                     tt=np.arange(0,T+dt,dt)
#                     for i,t in enumerate(tt):
#                         if i==0:
#                             x_ex[i,:] = np.array([x0,y0])
#                         else:
#                             x_ex[i,0] = x_ex[i-1,0] + dt*x_ex[i-1,0]*(alpha-beta*x_ex[i-1,1]-rho*x_ex[i-1,0])
#                             x_ex[i,1] = x_ex[i-1,1] + dt*x_ex[i-1,1]*(-gamma+delta*x_ex[i-1,0])
#                         if i>=7:
#                             input_list.append(x_ex[i-7:i,:].T)
#                             output_list.append(x_ex[i,:].T)

# input_train = np.array(input_list)
# output_train = np.array(output_list)


In [16]:
T = 100
Nt = 100
dt = T/Nt
x0_v = np.arange(0.1, 1, 0.1)
y0_v = np.arange(0.1, 1, 0.1)
rho = 0.1
alpha = 0.25
beta = 0.2
gamma = 0.1
delta = 0.1

input_list = []
output_list =[]

for x0 in x0_v:
    for y0 in y0_v:
        x_ex = np.zeros((Nt+1,2))
        tt=np.arange(0,T+dt,dt)
        for i,t in enumerate(tt):
            if i==0:
                x_ex[i,:] = np.array([x0,y0])
            else:
                x_ex[i,0] = x_ex[i-1,0] + dt*x_ex[i-1,0]*(alpha-beta*x_ex[i-1,1]-rho*x_ex[i-1,0])
                x_ex[i,1] = x_ex[i-1,1] + dt*x_ex[i-1,1]*(-gamma+delta*x_ex[i-1,0])
            if i>=1:
                input_list.append(x_ex[i-1:i,:].T)
                output_list.append(x_ex[i,:].T)

input_train = np.array(input_list)
output_train = np.array(output_list)

In [17]:
save_flag = False
save_flag = True
if save_flag:
    np.save('../data/input_train.npy', input_train)
    np.save('../data/output_train.npy', output_train)

Test set

In [18]:
# T = 50
# Nt = 50
# dt = T/Nt
# x0 = 1
# y0 = 0.1
# rho_v = np.arange(0.005, 0.09, 0.02)
# alpha_v = np.arange(0.005, 0.09, 0.02)
# beta_v = np.arange(0.005, 0.09, 0.02)
# gamma_v = np.arange(0.005, 0.09, 0.02)
# delta_v = np.arange(0.005, 0.09, 0.02)
# n_tot = rho_v.shape[0]**5
# k=0
# input_list = []
# output_list =[]

# for rho in rho_v:
#     for alpha in alpha_v:
#         for beta in beta_v:
#             for gamma in gamma_v:
#                 for delta in delta_v:
#                     k += 1
#                     # print('Advancing: '+str(k/n_tot*100)+'%')
#                     x_ex = np.zeros((Nt+1,2))
#                     tt=np.arange(0,T+dt,dt)
#                     for i,t in enumerate(tt):
#                         if i==0:
#                             x_ex[i,:] = np.array([x0,y0])
#                         else:
#                             x_ex[i,0] = x_ex[i-1,0] + dt*x_ex[i-1,0]*(alpha-beta*x_ex[i-1,1]-rho*x_ex[i-1,0])
#                             x_ex[i,1] = x_ex[i-1,1] + dt*x_ex[i-1,1]*(-gamma+delta*x_ex[i-1,0])
#                         if i>=7:
#                             input_list.append(x_ex[i-7:i,:].T)
#                             output_list.append(x_ex[i,:].T)

# input_test = np.array(input_list)
# output_test = np.array(output_list)


In [19]:
T = 100
Nt = 100
dt = T/Nt
x0_v = np.arange(0.5, 1, 0.2)
y0_v = np.arange(0.5, 1, 0.2)
rho = 0.1
alpha = 0.25
beta = 0.2
gamma = 0.1
delta = 0.1

input_list = []
output_list =[]

for x0 in x0_v:
    for y0 in y0_v:
        x_ex = np.zeros((Nt+1,2))
        tt=np.arange(0,T+dt,dt)
        for i,t in enumerate(tt):
            if i==0:
                x_ex[i,:] = np.array([x0,y0])
            else:
                x_ex[i,0] = x_ex[i-1,0] + dt*x_ex[i-1,0]*(alpha-beta*x_ex[i-1,1]-rho*x_ex[i-1,0])
                x_ex[i,1] = x_ex[i-1,1] + dt*x_ex[i-1,1]*(-gamma+delta*x_ex[i-1,0])
            if i>=1:
                input_list.append(x_ex[i-1:i,:].T)
                output_list.append(x_ex[i,:].T)

input_test = np.array(input_list)
output_test = np.array(output_list)

In [21]:
save_flag = False
save_flag = True
if save_flag:
    np.save('../data/input_test.npy', input_test)
    np.save('../data/output_test.npy', output_test)
    print('Dataset saved')

Dataset saved
